In [81]:
import numpy as np
import cv2 as cv
import os

In [82]:
# read the sift vectors from the file
def read_sift_vectors(filename):
    sift_vectors = []
    x = 0
    # parse the file at paranthesis
    with open(filename, 'r') as f:
        # read first line of f
        line = f.readline()
        while line:
            temp_list = []
            line = line.strip()
            if line.startswith('['):
                while(not line.endswith(']\n')):
                    temp = line.replace('[', '')
                    temp = temp.replace(']', '')
                    temp = temp.replace('\n', '')
                    temp = temp[1:]
                    temp = temp.split(' ')
                    # remove empty strings
                    temp = list(filter(None, temp))
                    temp_list.extend(float(temp[i]) for i in range(len(temp)))
                    line = f.readline()
                temp = line.replace(']', '') 
                temp = temp.replace('\n', '')
                temp = temp[1:]
                temp = temp.split(' ')
                # remove empty strings
                temp = list(filter(None, temp))
                temp_list.extend(float(temp[i]) for i in range(len(temp)))


            # convert to numpy array
            # parse the list to get the float values
            temp_list = [float(i) for i in temp_list]
            temp_list = np.array(temp_list)
            sift_vectors.append(temp_list)
            line = f.readline()



    return np.array(sift_vectors)

In [83]:
sift_file = './image_words_sift.txt'

# read the sift vectors
sift_vectors = read_sift_vectors(sift_file)

In [84]:
def getFeatureVectors_sift(img, vocabulary):
    features = np.zeros(len(vocabulary[0]))
    sift = cv.SIFT_create()
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    kp = []
    des = []
    k = 60
    for l in range(img.shape[0]//k):
        for j in range(img.shape[1]//k):
            kp_block = sift.detect(gray[l*k:(l+1)*k,j*k:(j+1)*k], None)
            # extract only the strongest keypoints in each block
            kp_block = sorted(kp_block, key=lambda x: -x.response)[:1]
            # if there is no keypoint in the block, take the centre point as the keypoint
            if len(kp_block) == 0:
                kp_block = [cv.KeyPoint(l*k+k/2,j*k+k/2,k)]
            kp_block,des_block = sift.compute(gray[l*k:(l+1)*k,j*k:(j+1)*k],kp_block)

            # find minimum distance between the descriptors of the block and the vocabulary
            features[np.argmin([np.linalg.norm(des_block - v) for v in vocabulary])] += 1

    return features

In [87]:
image = cv.imread('./imgs/25.jpg')

# get the feature vectors
feature_vectors = getFeatureVectors_sift(image, sift_vectors)